In [7]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from typing import Literal

In [8]:
class State(TypedDict):
    seed: int

graph_builder = StateGraph(State)

In [9]:
def node_first(state):
    print("node_first", state)
    return {}

def node_second(state):
    print("node_second", state)
    return {}

def node_third(state):
    print("node_third", state)
    return {}


def node_fourth(state):
    print("node_fourth", state)
    return {}

In [10]:
graph_builder.add_node("node_first", node_first)
graph_builder.add_node("node_second", node_second)
graph_builder.add_node("node_third", node_third)
graph_builder.add_node("node_fourth", node_fourth)

# def decide_path(state: State) -> Literal["node_third", "node_fourth"]:
#     if state["seed"] % 2 == 0:
#         return "node_third"
#     else:
#         return "node_fourth"

def decide_path(state: State):
    return state["seed"] % 2 == 0

graph_builder.add_edge(START, "node_first")
graph_builder.add_edge("node_first", "node_second")
graph_builder.add_conditional_edges(
    "node_second", 
    decide_path, 
    {
        True: "node_third", 
        False: "node_fourth",
        "hello": END,
     },
)
graph_builder.add_edge("node_third", END)
graph_builder.add_edge("node_fourth", END)

In [ ]:
graph = graph_builder.compile()

# graph

graph.invoke({"seed": 2})

node_first {'seed': 1}
node_second {'seed': 1}
node_fourth {'seed': 1}


{'seed': 1}